<br>
<br>

# **Modelos del lenguaje basados en redes neuronales artificiales**

## **Transformers**

https://jalammar.github.io/illustrated-transformer/

https://lena-voita.github.io/nlp_course/seq2seq_and_attention.html#transformer_intro


Los modelos Transformer son una clase de arquitecturas de aprendizaje profundo que han revolucionado el campo del procesamiento del lenguaje natural (NLP) y más allá. Introducidos en el paper "Attention Is All You Need" por Vaswani et al. en 2017, los Transformers se destacan por su capacidad para manejar secuencias de datos, como texto, de una manera muy eficiente y efectiva. A diferencia de las arquitecturas anteriores como las redes neuronales recurrentes (RNN) y las redes neuronales convolucionales (CNN), los Transformers se basan principalmente en un mecanismo llamado "atención", especialmente la "atención de múltiples cabezas".

Aquí hay algunos puntos clave sobre los modelos Transformer:

1. **Mecanismo de Atención**: El componente central de un Transformer es el mecanismo de atención, que permite al modelo ponderar la importancia de diferentes partes de la entrada. En el contexto del NLP, esto significa que un Transformer puede prestar más atención a palabras relevantes y menos a las irrelevantes al procesar un texto.

2. **Atención de Múltiples Cabezas**: Los Transformers utilizan lo que se llama "atención de múltiples cabezas". Esto les permite prestar atención a diferentes partes de la secuencia de entrada simultáneamente, lo que mejora la capacidad del modelo para aprender relaciones complejas.

3. **Arquitectura de Codificador y Decodificador**: Los Transformers originales se componen de una serie de bloques de codificadores y decodificadores. El codificador procesa la entrada (por ejemplo, un texto en un idioma fuente para la traducción automática), y el decodificador genera la salida (por ejemplo, el texto traducido). Cada bloque de codificador y decodificador contiene capas de atención y redes neuronales feed-forward.

4. **Paralelización y Escalabilidad**: A diferencia de las RNN, los Transformers no requieren que los datos se procesen secuencialmente. Esto significa que pueden manejar secuencias completas de datos a la vez, lo que permite una paralelización masiva y hace que los Transformers sean particularmente adecuados para el hardware moderno de GPU.

5. **Aplicaciones en NLP y Más Allá**: Aunque los Transformers fueron diseñados originalmente para tareas de NLP como la traducción automática, su aplicación se ha extendido a una amplia gama de tareas, incluyendo la generación de texto (como GPT-3), el entendimiento del lenguaje (como BERT), el procesamiento de imágenes, y más.

6. **Transferencia y Aprendizaje Pre-entrenado**: Modelos como BERT y GPT-3 son pre-entrenados en grandes cantidades de datos y luego afinados en tareas específicas, lo que les permite alcanzar un rendimiento destacado en una amplia gama de tareas de NLP con relativamente poco esfuerzo de ajuste específico de la tarea.




---

## **Práctica 3: Implementación del mecanismo de auto-atención del modelo Transformer**

Suponiendo que tenemos los vectores Q, K y V, implementa el mecanismo de auto-atención del modelo Transformer.


In [1]:
import torch

Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])



#### **Objetivos de la práctica**

- Entender con detalle el funcionamiento del mecanismo de auto-atención.
- Practicar las operaciones matriciales en PyTorch.


Una vez realizado el resultado debe ser:

<table>
<tr>
<td><b>Attn 1</b></td><td>0.2500</td><td>0.5000</td><td>0.5000</td>
</tr>
<tr>
<td><b>Attn 2</b></td><td>0.1892</td><td>0.5432</td><td>0.5857</td>
</tr>
<tr>
<td><b>Attn 3</b></td><td>0.2372</td><td>0.5087</td><td>0.5173</td>
</tr>
<tr>
<td><b>Attn 4</b></td><td>0.2309</td><td>0.5130</td><td>0.5260</td>
</tr>
</table>

---

